<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

[https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches](https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/Falcon9\_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing\_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/falcon9-launches-wiki.png)


## Objectives

Web scrap Falcon 9 launch records with `BeautifulSoup`:

*   Extract a Falcon 9 launch records HTML table from Wikipedia
*   Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [7]:
# use requests.get() method with the provided static_url
# assign the response to a object
response=requests.get(static_url)


In [8]:
response.status_code

200

Create a `BeautifulSoup` object from the HTML `response`


In [150]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup=BeautifulSoup(response.content,'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly


In [119]:
# Use soup.title attribute
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [120]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables=soup.find_all('table')

Starting from the third table is our target table contains the actual launch records.


In [133]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table.prettify())

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    time (
    )
   </th>
   <th scope="col">
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
   </th>
   <th scope="col">
    Payload mass
   </th>
   <th scope="col">
    Orbit
   </th>
   <th scope="col">
    Customer
   </th>
   <th scope="col">
    Launch
    outcome
   </th>
   <th scope="col">
   </th>
  </tr>
  <tr>
   <th rowspan="2" scope="row" style="text-align:center;">
    1
   </th>
   <td>
    4 June 2010,
    <br/>
    18:45
   </td>
   <td>
    <a href="/wiki/Falcon_9_v1.0" title="Falcon 9 v1.0">
     F9 v1.0
    </a>
    <sup class="reference" id="cite_ref-MuskMay2012_13-0">
     <a href="#cite_note-MuskMay2012-13">
      [7]
     </a>
    </sup>
    <br/>
    B0003.1
    <sup class="reference" id="cite_ref-block_numbers_14-0">
     <a href="#cite_note-block_numbers

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [122]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
temp=first_launch_table.find_all('th')

In [123]:
for x in range(len(temp)):
    try:
        name=extract_column_from_header(temp[x])
        if (name is not None and len(name)>0):
            column_names.append(name)
    except:
        pass

Check the extracted column names


In [124]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [226]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [227]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'].append(flight_number)
            datatimelist=date_time(row[0])
            #print(flight_number)
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            #print(date)
                        
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            #print(bv)
            
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            print(payload)
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            #print(orbit)

            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            #customer = row[6].a.string
            customer=row[6].text.strip()
            launch_dict['Customer'].append(customer)
            #print(customer)
            
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            #print(booster_landing)
            

Dragon Spacecraft Qualification Unit
Dragon
Dragon
SpaceX CRS-1
SpaceX CRS-2
CASSIOPE
SES-8
Thaicom 6
SpaceX CRS-3
Orbcomm-OG2
AsiaSat 8
AsiaSat 6
SpaceX CRS-4
SpaceX CRS-5
DSCOVR
ABS-3A
SpaceX CRS-6
TürkmenÄlem 52°E / MonacoSAT
SpaceX CRS-7
Orbcomm-OG2
Jason-3
SES-9
SpaceX CRS-8
JCSAT-14
Thaicom 8
ABS-2A
SpaceX CRS-9
JCSAT-16
Iridium NEXT
SpaceX CRS-10
EchoStar 23
SES-10
NROL-76
Inmarsat-5 F4
SpaceX CRS-11
BulgariaSat-1
Iridium NEXT
Intelsat 35e
SpaceX CRS-12
Formosat-5
Boeing X-37B
Iridium NEXT
SES-11
Koreasat 5A
SpaceX CRS-13
Iridium NEXT
Zuma
GovSat-1
Paz
Hispasat 30W-6
Iridium NEXT
SpaceX CRS-14
Transiting Exoplanet Survey Satellite
Bangabandhu-1
Iridium NEXT
SES-12
SpaceX CRS-15
Telstar 19V
Iridium NEXT
Merah Putih
Telstar 18V
SAOCOM 1A
Es'hail 2
SSO-A
SpaceX CRS-16
GPS III
Iridium NEXT
Nusantara Satu
Crew Dragon Demo-1
SpaceX CRS-17
Starlink
RADARSAT Constellation
SpaceX CRS-18
AMOS-17
Starlink
SpaceX CRS-19
JCSat-18
Starlink
Crew Dragon in-flight abort test
Starlink
Starlink
Sp

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [228]:
df=pd.DataFrame(launch_dict)

In [229]:
df.shape

(121, 11)

In [233]:
df.tail(20)

Flight No. Launch site        Payload Payload mass Orbit  \
101        102       CCSFS          SXM-7     7,000 kg   GTO   
102        103         KSC       NROL-108            C   LEO   
103        104       CCSFS     Türksat 5A     3,500 kg   GTO   
104        105         KSC       Starlink    15,600 kg   LEO   
105        106       CCSFS  Transporter-1    ~5,000 kg   SSO   
106        107       CCSFS       Starlink    15,600 kg   LEO   
107        108       CCSFS       Starlink    15,600 kg   LEO   
108        109         KSC       Starlink    15,600 kg   LEO   
109        110       CCSFS       Starlink    15,600 kg   LEO   
110        111         KSC       Starlink    15,600 kg   LEO   
111        112       CCSFS       Starlink    15,600 kg   LEO   
112        113       CCSFS       Starlink    15,600 kg   LEO   
113        114         KSC         Crew-2   ~13,000 kg   LEO   
114        115       CCSFS       Starlink    15,600 kg   LEO   
115        116         KSC       Starlink    15,600 kg   LEO   
116        117       CCSFS       Starlink    15,600 kg   LEO   
117        118         KSC       Starlink   ~14,000 kg   LEO   
118        119       CCSFS       Starlink    15,600 kg   LEO   
119        120         KSC  SpaceX CRS-22     3,328 kg   LEO   
120        121       CCSFS          SXM-8     7,000 kg   GTO   

                           Customer Launch outcome Version Booster  \
101                       Sirius XM      Success\n         F9 B5 ♺   
102                             NRO      Success\n         F9 B5 ♺   
103                         Türksat      Success\n           F9 B5   
104                          SpaceX      Success\n    F9 B5B1051.8   
105                         Various      Success\n    F9 B5B1058.5   
106                          SpaceX      Success\n         F9 B5 ♺   
107                          SpaceX      Success\n         F9 B5 ♺   
108                          SpaceX      Success\n         F9 B5 ♺   
109                          SpaceX      Success\n         F9 B5 ♺   
110                          SpaceX      Success\n         F9 B5 ♺   
111                          SpaceX      Success\n    F9 B5B1060.6   
112                          SpaceX      Success\n         F9 B5 ♺   
113                 NASA (CTS)[497]      Success\n    F9 B5B1061.2   
114                          SpaceX      Success\n    F9 B5B1060.7   
115                          SpaceX      Success\n    F9 B5B1049.9   
116                          SpaceX      Success\n   F9 B5B1051.10   
117  SpaceX Capella Space and Tyvak      Success\n    F9 B5B1058.8   
118                          SpaceX      Success\n    F9 B5B1063.2   
119                      NASA (CRS)      Success\n    F9 B5B1067.1   
120                       Sirius XM      Success\n           F9 B5   

    Booster landing              Date      Time  
101         Success  13 December 2020  17:30:00  
102         Success  19 December 2020  14:00:00  
103         Success    8 January 2021     02:15  
104         Success   20 January 2021     13:02  
105         Success   24 January 2021     15:00  
106         Success   4 February 2021     06:19  
107         Failure  16 February 2021  03:59:37  
108         Success      4 March 2021     08:24  
109         Success     11 March 2021  08:13:29  
110         Success     14 March 2021     10:01  
111         Success     24 March 2021     08:28  
112         Success      7 April 2021     16:34  
113         Success     23 April 2021      9:49  
114         Success     29 April 2021     03:44  
115         Success        4 May 2021     19:01  
116         Success        9 May 2021     06:42  
117         Success       15 May 2021     22:56  
118         Success       26 May 2021     18:59  
119         Success       3 June 2021     17:29  
120         Success       6 June 2021     04:26

In [232]:
df.to_csv('spacex_web_scraped.csv', index=False)


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates               |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
